**GETTING THE DATA**


In [1]:
import requests
import pandas as pd
import time

In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

from src import config
WEBTOON_DATA_PATH = config.WEBTOON_DATA_PATH

In [3]:
URL = "https://graphql.anilist.co"

In [4]:
QUERY = """
query ($page: Int) {
  Page(page: $page, perPage: 50) {
    media(type: MANGA, sort: POPULARITY_DESC) {
      id
      title {
        romaji
        english
      }
      format
      chapters
      volumes
      genres
      tags {
        name
      }
      countryOfOrigin
      popularity
      favourites
      averageScore
      status
      description
    }
  }
}
"""


In [5]:
session = requests.Session()
session.headers.update({"Content-Type": "application/json"})

In [6]:
all_data = []
TOTAL_PAGES = 400 

In [7]:
def safe_get(d, key, default=None):
    return d[key] if d and key in d and d[key] is not None else default

In [8]:
print("TURBO SCRAPER BAŞLADI...\n")

for page in range(1, TOTAL_PAGES + 1):
    success = False

    for attempt in range(3):
        try:
            response = session.post(
                URL,
                json={"query": QUERY, "variables": {"page": page}},
                timeout=10
            )

            # Too Many Requests kontrolü
            if response.status_code == 429:
                print(f"Sayfa {page}: Rate Limit! 5 saniye bekleniyor...")
                time.sleep(5)
                continue

            # Başka hata durumu
            if response.status_code != 200:
                print(f"Sayfa {page}: Hata kodu {response.status_code}, tekrar deneniyor...")
                time.sleep(1)
                continue

            data = response.json().get("data", {}).get("Page", {}).get("media", [])
            if not data:
                print(f"Sayfa {page}: Boş veri — deneniyor...")
                time.sleep(1)
                continue

            # Veri işleme
            for m in data:
                all_data.append({
                    "id": m.get("id"),
                    "title_romaji": safe_get(m.get("title"), "romaji"),
                    "title_english": safe_get(m.get("title"), "english"),
                    "format": m.get("format"),
                    "chapters": m.get("chapters"),
                    "volumes": m.get("volumes"),
                    "genres": m.get("genres", []),
                    "tags": [t.get("name") for t in m.get("tags", [])],
                    "country": m.get("countryOfOrigin"),
                    "popularity": m.get("popularity"),
                    "favourites": m.get("favourites"),
                    "averageScore": m.get("averageScore"),
                    "status": m.get("status"),
                    "description": m.get("description"),
                })

            print(f"✓ Sayfa {page}/{TOTAL_PAGES} — Toplam: {len(all_data)}")
            success = True
            break

        except Exception as e:
            print(f"Sayfa {page} hata (deneme {attempt+1}): {e}")
            time.sleep(1)

    if not success:
        print(f"Sayfa {page} tamamen atlandı.")
        continue

    time.sleep(0.35)

# Veri kaydı
df = pd.DataFrame(all_data)
df.to_csv(WEBTOON_DATA_PATH, index=False, encoding="utf-8-sig")

print("\nTAMAMLANDI!")
print("Toplam veri:", len(df))

TURBO SCRAPER BAŞLADI...

✓ Sayfa 1/400 — Toplam: 50
✓ Sayfa 2/400 — Toplam: 100
✓ Sayfa 3/400 — Toplam: 150
✓ Sayfa 4/400 — Toplam: 200
✓ Sayfa 5/400 — Toplam: 250
✓ Sayfa 6/400 — Toplam: 300
✓ Sayfa 7/400 — Toplam: 350
✓ Sayfa 8/400 — Toplam: 400
✓ Sayfa 9/400 — Toplam: 450
✓ Sayfa 10/400 — Toplam: 500
✓ Sayfa 11/400 — Toplam: 550
✓ Sayfa 12/400 — Toplam: 600
✓ Sayfa 13/400 — Toplam: 650
✓ Sayfa 14/400 — Toplam: 700
✓ Sayfa 15/400 — Toplam: 750
✓ Sayfa 16/400 — Toplam: 800
✓ Sayfa 17/400 — Toplam: 850
✓ Sayfa 18/400 — Toplam: 900
✓ Sayfa 19/400 — Toplam: 950
Sayfa 20: Rate Limit! 5 saniye bekleniyor...
Sayfa 20: Rate Limit! 5 saniye bekleniyor...
✓ Sayfa 20/400 — Toplam: 1000
✓ Sayfa 21/400 — Toplam: 1050
✓ Sayfa 22/400 — Toplam: 1100
✓ Sayfa 23/400 — Toplam: 1150
✓ Sayfa 24/400 — Toplam: 1200
✓ Sayfa 25/400 — Toplam: 1250
✓ Sayfa 26/400 — Toplam: 1300
✓ Sayfa 27/400 — Toplam: 1350
✓ Sayfa 28/400 — Toplam: 1400
✓ Sayfa 29/400 — Toplam: 1450
✓ Sayfa 30/400 — Toplam: 1500
✓ Sayfa 31/40

In [11]:
df.to_json(r"C:\Users\aleyna nur\Desktop\Manga_Manhwa_Manhua_Rec\datasets\raw\webtoon_data.json", orient="records", force_ascii=False, indent=2)